In [449]:
# Стандартные библиотеки
import pandas as pd
import requests

# Сторонние библиотеки
import datetime
import re


# Содержание
* [1. Данные по ипотечным ставкам](#chapter1)
* [2. Данные по ценам на недвижимость](#chapter2)
* [3. Макроэкономические данные](#chapter3)
* [4. Предобработка данных](#chapter4)

## 1. Данные по ипотечным ставкам <a class="anchor" id="chapter1"></a>

Источник: ЦБ РФ  
Период: 2013–2024 (10 лет)

- Ключевая ставка ЦБ РФ (%) - https://cbr.ru/hd_base/KeyRate/
- Средний уровень процентной ставки по ипотечному кредиту, региональный разрез (%) - https://www.fedstat.ru/indicator/59319
- Средневзвешенная процентная ставка по ипотечным жилищным кредитам в рублях, региональный разрез (%) - https://www.fedstat.ru/indicator/60293 


In [450]:
start_date =  datetime.date(2010, 1, 1).strftime('%d.%m.%Y')
end_date =  datetime.date(2024, 12, 31).strftime('%d.%m.%Y')

# Ключевая ставка 
url_key_rate = f'https://cbr.ru/hd_base/KeyRate/?UniDbQuery.Posted=True&UniDbQuery.From={start_date}&UniDbQuery.To={end_date}'

response = requests.get(url_key_rate)
response.raise_for_status()

key_rate_df = pd.read_html(url_key_rate)[0]

key_rate_df.columns = ['date', 'key_rate']
key_rate_df['key_rate'] = key_rate_df['key_rate']/100

key_rate_df

,date,key_rate
0,30.12.2024,21.0
1,28.12.2024,21.0
2,27.12.2024,21.0
3,26.12.2024,21.0
4,25.12.2024,21.0
...,...,...
2821,23.09.2013,5.5
2822,20.09.2013,5.5
2823,19.09.2013,5.5
2824,18.09.2013,5.5


In [451]:
# Средний уровень процентной ставки по ипотечному кредиту, региональный разрез (%)

file_path = r'..\data\excel\home_average_mortgage_rate_region.xls'
home_average_mortgage_rate_df = pd.read_excel(file_path, sheet_name='Данные', skiprows=2)
home_average_mortgage_rate_df.columns = ['region','year', 'month', 'avg_rate']

home_average_mortgage_rate_df


,region,year,month,avg_rate
0,Российская Федерация,2018,значение показателя за год,9.56
1,Российская Федерация,2019,январь,9.88
2,Российская Федерация,2019,февраль,10.04
3,Российская Федерация,2019,март,10.18
4,Российская Федерация,2019,апрель,10.28
...,...,...,...,...
7487,Чукотский автономный округ,2024,август,8.38
7488,Чукотский автономный округ,2024,сентябрь,8.57
7489,Чукотский автономный округ,2024,октябрь,8.86
7490,Чукотский автономный округ,2024,ноябрь,8.85


In [452]:
# Средневзвешенная процентная ставка по ипотечным кредитам, региональный разрез (%)

file_path = r'..\data\excel\home_weight_avg_mortgage_rate_region.xls'
home_weight_avg_mortgage_rate_df = pd.read_excel(file_path, sheet_name='Данные', skiprows=2)
home_weight_avg_mortgage_rate_df.columns = ['region','year', 'month', 'weight_avg_rate']

home_weight_avg_mortgage_rate_df

,region,year,month,weight_avg_rate
0,Российская Федерация,2018,значение показателя за год,9.56
1,Российская Федерация,2019,январь,9.88
2,Российская Федерация,2019,февраль,10.04
3,Российская Федерация,2019,март,10.18
4,Российская Федерация,2019,апрель,10.28
...,...,...,...,...
7487,Чукотский автономный округ,2024,август,8.38
7488,Чукотский автономный округ,2024,сентябрь,8.57
7489,Чукотский автономный округ,2024,октябрь,8.86
7490,Чукотский автономный округ,2024,ноябрь,8.85


In [453]:
# TODO: нет столбца даты
# Ипотечные жилищные кредиты, предоставленные физическим лицам-резидентам,
# и приобретенные права требования по ипотечным жилищным кредитам

xlsx_file_path = r'..\data\excel\02_02_Mortgage.xlsx'

mortgage_df = pd.read_excel(xlsx_file_path, sheet_name='в рублях', skiprows=1)
mortgage_df = mortgage_df.set_index('Отчетная дата').T
mortgage_df.columns = [
    'loans_issued_monthly_count',
    'loans_issued_monthly_volume_millions_rub',
    'total_debt_millions_rub',
    'overdue_debt_millions_rub',
    'acquired_mortgage_rights_debt_millions_rub',
    'total_debt_including_acquired_rights_millions_rub',
    'weighted_average_loan_term_months',
    'weighted_average_interest_rate_percent'
]

# Добавим новый признак 'Тип ипотеки' и разбиение 'новостройка = 1', 'вторичное жилье = 0'
mortgage_df['mortgage_type'] = 0

# Ипотечные жилищные кредиты, предоставленные физическим лицам-резидентам 
# под залог прав требования по договорам участия в долевом строительстве (ДДУ), 
# и приобретенные права требования по ипотечным жилищным кредитам под залог ДДУ

xlsx_file_path = r'..\data\excel\02_03_Scpa_mortgage.xlsx'

scpa_mortgage_df = pd.read_excel(xlsx_file_path, sheet_name='в рублях', skiprows=[0, 10])
scpa_mortgage_df = scpa_mortgage_df.set_index('Отчетная дата').T
scpa_mortgage_df.columns = [
    'loans_issued_monthly_count',
    'loans_issued_monthly_volume_millions_rub',
    'total_debt_millions_rub',
    'overdue_debt_millions_rub',
    'acquired_mortgage_rights_debt_millions_rub',
    'total_debt_including_acquired_rights_millions_rub',
    'weighted_average_loan_term_months',
    'weighted_average_interest_rate_percent'
]

# Добавим новый признак 'Тип ипотеки' и разбиение 'новостройка = 1', 'вторичное жилье = 0'
scpa_mortgage_df['mortgage_type'] = 1

# Совместим два датафрейма
home_info_mortgage_df = pd.concat([mortgage_df, scpa_mortgage_df], ignore_index=True)

home_info_mortgage_df


,loans_issued_monthly_count,loans_issued_monthly_volume_millions_rub,total_debt_millions_rub,overdue_debt_millions_rub,acquired_mortgage_rights_debt_millions_rub,total_debt_including_acquired_rights_millions_rub,weighted_average_loan_term_months,weighted_average_interest_rate_percent,mortgage_type
0,151308.0,290469.0,5144935.0,54575.0,134733.0,5279668.0,187.6,9.79,0
1,78043.0,148275.0,5184710.0,55965.0,133665.0,5318375.0,191.1,9.85,0
2,102654.0,198940.0,5272293.0,56636.0,138745.0,5411038.0,190.0,9.75,0
3,118702.0,234757.0,5381205.0,56469.0,139759.0,5520964.0,193.4,9.63,0
4,125300.0,249267.0,5500335.0,57186.0,138488.0,5638823.0,189.5,9.6,0
...,...,...,...,...,...,...,...,...,...
165,28939.0,161077.0,4758000.0,7486.0,161071.0,4919071.0,322.9,6.35,1
166,32053.0,179703.0,4730537.0,7951.0,158638.0,4889175.0,323.9,6.25,1
167,32612.0,185279.0,4687581.0,8564.0,200321.0,4887903.0,323.4,6.16,1
168,30053.0,160708.0,4639490.0,8792.0,200296.0,4839787.0,321.2,6.06,1


## 2. Данные по ценам на недвижимость <a class="anchor" id="chapter2"></a>

Источник: Росстат, Kaggle, порталы недвижимости (ЦИАН, Avito), Домклик   
Период: 2014-2024

- Средняя цена за квадратный метр (руб) -  https://www.fedstat.ru/indicator/31452
- Индекс цен на рынке жилья по регионам , региональный разрез (%) - https://www.fedstat.ru/indicator/30925


In [454]:
# Средняя цена 1 кв. м общей площади квартир на рынке жилья  (рубль)

file_path = r'..\data\excel\home_average_price_region.xls'
home_average_price_df = pd.read_excel(file_path, sheet_name='Данные', skiprows=2)
home_average_price_df.columns = ['type_of_housing_market','region','year', 'quarter', 'price']

home_average_price_df

,type_of_housing_market,region,year,quarter,price
0,Вторичный рынок жилья,Российская Федерация,2014,I квартал,55924.70
1,Вторичный рынок жилья,Российская Федерация,2014,II квартал,56630.42
2,Вторичный рынок жилья,Российская Федерация,2014,III квартал,57119.36
3,Вторичный рынок жилья,Российская Федерация,2014,IV квартал,58084.71
4,Вторичный рынок жилья,Российская Федерация,2015,I квартал,58707.41
...,...,...,...,...,...
8029,Первичный рынок жилья,Чукотский автономный округ,2024,III квартал,233799.46
8030,Первичный рынок жилья,Крымский федеральный округ,2016,I квартал,51250.42
8031,Первичный рынок жилья,Крымский федеральный округ,2016,II квартал,50877.34
8032,Первичный рынок жилья,Крымский федеральный округ,2016,III квартал,51239.18


In [455]:
# Индексы цен на рынке жилья (%)

file_path = r'..\data\excel\home_index_price_region.xls'
home_index_price_df = pd.read_excel(file_path, sheet_name='Данные', skiprows=3)
home_index_price_df.columns = ['type_of_housing_market', 'region', 'year', 'quarter', 'index']

home_index_price_df

,type_of_housing_market,region,year,quarter,index
0,Вторичный рынок жилья,Российская Федерация,2014,I квартал,103.64
1,Вторичный рынок жилья,Российская Федерация,2014,II квартал,103.53
2,Вторичный рынок жилья,Российская Федерация,2014,III квартал,104.12
3,Вторичный рынок жилья,Российская Федерация,2014,IV квартал,105.12
4,Вторичный рынок жилья,Российская Федерация,2015,I квартал,104.80
...,...,...,...,...,...
7839,Первичный рынок жилья,Чукотский автономный округ,2023,II квартал,100.00
7840,Первичный рынок жилья,Чукотский автономный округ,2023,III квартал,100.00
7841,Первичный рынок жилья,Чукотский автономный округ,2023,IV квартал,100.00
7842,Первичный рынок жилья,Чукотский автономный округ,2024,I квартал,112.77


## 3. Макроэкономические данные <a class="anchor" id="chapter3"></a>

Источник: Росстат, ЦБ РФ  
Период: 2014-2024

- Уровень инфляции (%) - https://cbr.ru/statistics/ddkp/infl/?utm_source=w&utm_content=page
- Среднедушевые денежные доходы населения, региональный разрез (руб)
- Курс доллара (руб)


In [456]:
start_date =  datetime.date(2014, 1, 1).strftime('%d.%m.%Y')
end_date =  datetime.date(2024, 12, 31).strftime('%d.%m.%Y')

# Ключевая ставка + Инфляция

url = f'https://cbr.ru/statistics/ddkp/infl/?UniDbQuery.Posted=True&UniDbQuery.From={start_date}&UniDbQuery.To={end_date}'

response = requests.get(url)
response.raise_for_status()

key_rate_inflation_df = pd.read_html(url)[0]

key_rate_inflation_df.columns = ['date', 'key_rate', 'inflation', 'target_inflation']
key_rate_inflation_df['key_rate'] = key_rate_inflation_df['key_rate']/100
key_rate_inflation_df['inflation'] = key_rate_inflation_df['inflation']/100
key_rate_inflation_df = key_rate_inflation_df.drop('target_inflation', axis=1)

key_rate_inflation_df

,date,key_rate,inflation
0,12.2024,21.0,9.52
1,11.2024,21.0,8.88
2,10.2024,21.0,8.54
3,9.2024,19.0,8.63
4,8.2024,18.0,9.05
...,...,...,...
127,5.2014,7.5,7.59
128,4.2014,7.5,7.33
129,3.2014,7.0,6.92
130,2.2014,5.5,6.21


In [457]:
# Среднедушевые денежные доходы населения

file_path = r'..\data\excel\avg_monetary_income_people_region.xlsx'

avg_monetary_income_people_df = pd.read_excel(file_path, skiprows=2)
avg_monetary_income_people_df = avg_monetary_income_people_df.drop([97, 98, 99, 100, 101])

# Таблица представлена не осень удобном формате, поэтому обработаем ее
collection_df_list = []

for i in range(1, len(avg_monetary_income_people_df)):   
    selected_rows = avg_monetary_income_people_df.iloc[[0, i], :]
    selected_rows = selected_rows.transpose()
    region = selected_rows.iloc[0, 1]
    selected_rows = selected_rows.drop('Unnamed: 0')
    selected_rows = selected_rows.reset_index()
    selected_rows['region'] = region
    selected_rows.columns = ['year', 'quarter', 'income', 'region']
    collection_df_list.append(selected_rows)

avg_monetary_income_people_df = pd.concat(collection_df_list, axis=0, ignore_index=True)
avg_monetary_income_people_df['year'] = avg_monetary_income_people_df['year'].replace(r'Unnamed:*', pd.NA, regex=True)
avg_monetary_income_people_df['year'] = avg_monetary_income_people_df['year'].ffill()

avg_monetary_income_people_df.info()
avg_monetary_income_people_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6144 entries, 0 to 6143
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   year     6144 non-null   object
 1   quarter  6048 non-null   object
 2   income   6112 non-null   object
 3   region   6144 non-null   object
dtypes: object(4)
memory usage: 192.1+ KB


,year,quarter,income,region
0,2011\nгод,NaN,20771.642488,Российская Федерация
1,2012 год,I кв.,19105.67146,Российская Федерация
2,2012 год,II кв.,22572.96167,Российская Федерация
3,2012 год,III кв.,23262.024407,Российская Федерация
4,2012 год,IV кв.,27963.833212,Российская Федерация
...,...,...,...,...
6139,2023 год,IV кв. 2),156092,Чукотский авт.округ
6140,2023 год,год 3),156988,Чукотский авт.округ
6141,2024 год 2),I кв.,147290,Чукотский авт.округ
6142,2024 год 2),II кв.,157555,Чукотский авт.округ


In [458]:
# Курс доллара
start_date =  datetime.date(2014, 1, 1).strftime('%d/%m/%Y')
end_date =  datetime.date(2024, 12, 31).strftime('%d/%m/%Y')

xml_url = f'http://www.cbr.ru/scripts/XML_dynamic.asp?date_req1={start_date}&date_req2={end_date}&VAL_NM_RQ=R01235'

us_rate_df = pd.read_xml(xml_url)

us_rate_df

,Date,Id,Nominal,Value,VunitRate
0,01.01.2014,R01235,1,"32,6587","32,6587"
1,10.01.2014,R01235,1,"33,1547","33,1547"
2,11.01.2014,R01235,1,"33,2062","33,2062"
3,14.01.2014,R01235,1,"33,1204","33,1204"
4,15.01.2014,R01235,1,"33,2386","33,2386"
...,...,...,...,...,...
2710,25.12.2024,R01235,1,"99,8729","99,8729"
2711,26.12.2024,R01235,1,"99,6125","99,6125"
2712,27.12.2024,R01235,1,"99,2295","99,2295"
2713,28.12.2024,R01235,1,"100,5281","100,5281"


In [459]:
# TODO: спарсить ипотечную ставку с сайта о недвижимости по типу ДомКлик
# так как с сайта цб подозрительно маленькая ставка

## 4. Предобработка данных <a class="anchor" id="chapter4"></a>

### Этапы предобработки данных:
- **Приведение столбцов с датами к единому формату**  
- **Удаление шума, исправление ошибок и устранение некорректных значений**  
- **Объединение данных из разных источников в единый набор**  
- **Преобразование данных в формат CSV для дальнейшей обработки**  

### Текущие наборы данных:
- **key_rate_df** — Ключевая ставка (%)
- **home_average_mortgage_rate_df** — Средний уровень процентной ставки по ипотечному кредиту, региональный разрез (%)
- **home_weight_avg_mortgage_rate_df** — Средневзвешенная процентная ставка по ипотечным кредитам, региональный разрез (%)
- **home_average_price_df** — Средняя цена 1 кв. м общей площади квартир на рынке жилья, региональный разрез (рубль)
- **home_index_price_df** — Индексы цен на рынке жилья, региональный разрез (%)
- **key_rate_inflation_df** — Инфляция (%)
- **avg_monetary_income_people_df** — Среднедушевые денежные доходы населения, региональный разрез (рубль)
- **us_rate_df** — Курс доллара, стоимость 1 доллара (рубль)



In [460]:
# комноненты, необходимые для предобработки
def datetime_year_month_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Преобразует DataFrame:
    - Удаляет строки с 'значение показателя за год'
    - Создаёт столбец 'date' в формате datetime64[ns] с использованием словаря месяцев
    - Удаляет столбцы 'year' и 'month'
    
    Args:
        df (pd.DataFrame): Исходный DataFrame с колонками 'year', 'month'
    
    Returns:
        pd.DataFrame: Обновлённый DataFrame с новым столбцом 'date'
    """

    month_dict = {
        'январь': '01', 'февраль': '02', 'март': '03', 'апрель': '04', 'май': '05',
        'июнь': '06', 'июль': '07', 'август': '08', 'сентябрь': '09', 'октябрь': '10',
        'ноябрь': '11', 'декабрь': '12'
    }
    df = df[df['month'] != 'значение показателя за год'].copy()
    df.loc[:, 'date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].map(month_dict) + '-01')
    df = df.drop(columns=['year', 'month'])

    return df

def datetime_year_quarter_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Преобразует DataFrame:
    - Создаёт столбец 'date' в формате datetime64[ns], используя year и quarter
    - Удаляет столбцы 'year' и 'quarter'
    
    Args:
        df (pd.DataFrame): Исходный DataFrame с колонками 'year', 'quarter'
    
    Returns:
        pd.DataFrame: Обновлённый DataFrame с новым столбцом 'date'
    """

    quarter_dict = {
        'I квартал': '01', 'II квартал': '04', 'III квартал': '07', 'IV квартал': '10'
    }
    df = df.copy()
    df.loc[:, 'date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['quarter'].map(quarter_dict) + '-01')
    df = df.drop(columns=['year', 'quarter'])

    return df

min_date_list = [] # Список самой ранней даты по каждому набору
max_date_list = [] # Список самой поздней даты по каждому набору

# предобработка # Ключевая ставка 
key_rate_df['date'] = pd.to_datetime(key_rate_df['date'], format='%d.%m.%Y')
key_rate_df.info()
min_date_list.append(min(key_rate_df['date']))
max_date_list.append(max(key_rate_df['date']))

# предобработка # Средний уровень процентной ставки по ипотечному кредиту, региональный разрез (%)
home_average_mortgage_rate_df = datetime_year_month_df(home_average_mortgage_rate_df)
home_average_mortgage_rate_df.info()
min_date_list.append(min(home_average_mortgage_rate_df['date']))
max_date_list.append(max(home_average_mortgage_rate_df['date']))

# предобработка # Средневзвешенная процентная ставка по ипотечным кредитам, региональный разрез (%)
home_weight_avg_mortgage_rate_df = datetime_year_month_df(home_weight_avg_mortgage_rate_df)
home_weight_avg_mortgage_rate_df.info()
min_date_list.append(min(home_weight_avg_mortgage_rate_df['date']))
max_date_list.append(max(home_weight_avg_mortgage_rate_df['date']))

# предобработка # Средняя цена 1 кв. м общей площади квартир на рынке жилья (рубль) 
home_average_price_df = datetime_year_quarter_df(home_average_price_df)
home_average_price_df.drop(columns=['type_of_housing_market'], inplace=True)
home_average_price_df.info()
min_date_list.append(min(home_average_price_df['date']))
max_date_list.append(max(home_average_price_df['date']))

# предобработка # Индексы цен на рынке жилья (%)
home_index_price_df = datetime_year_quarter_df(home_index_price_df)
home_index_price_df.drop(columns=['type_of_housing_market'], inplace=True)
home_index_price_df.info()
min_date_list.append(min(home_index_price_df['date']))
max_date_list.append(max(home_index_price_df['date']))

# предобработка # Ключевая ставка + Инфляция (%)
key_rate_inflation_df.drop(columns=['key_rate'], inplace=True)
key_rate_inflation_df['date'] = key_rate_inflation_df['date'].astype(str).str.replace(r'(\d{1,2})\.202\b', r'\1.2020', regex=True)
key_rate_inflation_df['date'] = pd.to_datetime(key_rate_inflation_df['date'], format='%m.%Y')
key_rate_inflation_df.info()
min_date_list.append(min(key_rate_inflation_df['date']))
max_date_list.append(max(key_rate_inflation_df['date']))

# предобработка # Среднедушевые денежные доходы населения (рубль) 
# очистка столбца "year"
avg_monetary_income_people_df['year'] = avg_monetary_income_people_df['year'].str[:4]
# очистка столбца 'quarter'
avg_monetary_income_people_df['quarter'] = avg_monetary_income_people_df['quarter'].str.extract(r'(IV|III|II|I)')
avg_monetary_income_people_df = avg_monetary_income_people_df.dropna(subset=['quarter'])
avg_monetary_income_people_df.loc[:, 'quarter']  = avg_monetary_income_people_df['quarter'] + ' квартал'

avg_monetary_income_people_df = datetime_year_quarter_df(avg_monetary_income_people_df)
avg_monetary_income_people_df.info()
min_date_list.append(min(avg_monetary_income_people_df['date']))
max_date_list.append(max(avg_monetary_income_people_df['date']))

# предобработка # Курс доллара
us_rate_df = us_rate_df[['Date', 'Value']].rename(columns={'Date': 'date', 'Value': 'us_rate'})
us_rate_df['date'] = pd.to_datetime(us_rate_df['date'], format='%d.%m.%Y')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2826 entries, 0 to 2825
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      2826 non-null   datetime64[ns]
 1   key_rate  2826 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 44.3 KB
<class 'pandas.core.frame.DataFrame'>
Index: 6912 entries, 1 to 7491
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   region    6912 non-null   object        
 1   avg_rate  6912 non-null   float64       
 2   date      6912 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 216.0+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 6912 entries, 1 to 7491
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   region           6912 non-null   o

In [461]:
# Выведем списки регионов и посмотрим если повторения
# Унифицируем регионы для каждого датасета и приведем их к единому формату для дальнейшего объединения
all_regions = pd.concat([
    home_average_mortgage_rate_df['region'],
    home_weight_avg_mortgage_rate_df['region'],
    home_average_price_df['region'],
    home_index_price_df['region'],
    avg_monetary_income_people_df['region']
]).unique()

print(all_regions)

['Российская Федерация' '    Центральный федеральный округ'
 '        Белгородская область' '        Брянская область'
 '        Владимирская область' '        Воронежская область'
 '        Ивановская область' '        Калужская область'
 '        Костромская область' '        Курская область'
 '        Липецкая область' '        Московская область'
 '        Орловская область' '        Рязанская область'
 '        Смоленская область' '        Тамбовская область'
 '        Тверская область' '        Тульская область'
 '        Ярославская область' '        г. Москва'
 '    Северо-Западный федеральный округ' '        Республика Карелия'
 '        Республика Коми' '        Архангельская область'
 '            в том числе Ненецкий автономный округ'
 '            Архангельская область без данных по Ненецкому автономному округу'
 '        Вологодская область' '        Калининградская область'
 '        Ленинградская область' '        Мурманская область'
 '        Новгородская область' '   

In [462]:
datasets = [
    home_average_mortgage_rate_df,
    home_weight_avg_mortgage_rate_df,
    home_average_price_df,
    home_index_price_df,
    avg_monetary_income_people_df
]

# убираем лишние пробелы в столбце 'region'
for df in datasets:
    df['region'] = df['region'].str.strip()  # Убираем начальные и конечные пробелы
    df['region'] = df['region'].str.replace(r'\s+', ' ', regex=True)  # Заменяем множественные пробелы на один

all_regions = pd.concat([
    home_average_mortgage_rate_df['region'],
    home_weight_avg_mortgage_rate_df['region'],
    home_average_price_df['region'],
    home_index_price_df['region'],
    avg_monetary_income_people_df['region']
]).unique()

print(all_regions)

['Российская Федерация' 'Центральный федеральный округ'
 'Белгородская область' 'Брянская область' 'Владимирская область'
 'Воронежская область' 'Ивановская область' 'Калужская область'
 'Костромская область' 'Курская область' 'Липецкая область'
 'Московская область' 'Орловская область' 'Рязанская область'
 'Смоленская область' 'Тамбовская область' 'Тверская область'
 'Тульская область' 'Ярославская область' 'г. Москва'
 'Северо-Западный федеральный округ' 'Республика Карелия'
 'Республика Коми' 'Архангельская область'
 'в том числе Ненецкий автономный округ'
 'Архангельская область без данных по Ненецкому автономному округу'
 'Вологодская область' 'Калининградская область' 'Ленинградская область'
 'Мурманская область' 'Новгородская область' 'Псковская область'
 'г. Санкт-Петербург'
 'Южный федеральный округ (до июля 2016г. Республика Крым и г.Севастополь входили в состав Крымского Федерального округа)'
 'Республика Адыгея (Адыгея)' 'Республика Калмыкия' 'Республика Крым'
 'Краснодарск

In [463]:
region_replace_dict = {
    "Российская Федерация": "Россия",
    "Город Москва столица Российской Федерации город федерального значения": "Москва",
    "г. Москва": "Москва",
    "г.Москва": "Москва",
    "Город Санкт-Петербург город федерального значения": "Санкт-Петербург",
    "г. Санкт-Петербург": "Санкт-Петербург",
    "г.Санкт-Петербург": "Санкт-Петербург",
    
    "Центральный федеральный округ": "ЦФО",
    "Северо-Западный федеральный округ": "СЗФО",
    "Южный федеральный округ": "ЮФО",

    "Южный федеральный округ (с 2010 года)": "ЮФО",
    "Южный федеральный округ (с 29.07.2016)": "ЮФО",
    "Южный федеральный округ (до июля 2016г. Республика Крым и г.Севастополь входили в состав Крымского Федерального округа)": "ЮФО",

    "Северо-Кавказский федеральный округ": "СКФО",
    "Приволжский федеральный округ": "ПФО",
    "Уральский федеральный округ": "УФО",
    "Сибирский федеральный округ": "СФО",
    "Сибирский федеральный округ (с 03.11.2018)": "СФО",
    "Сибирский федеральный округ 4)": "СФО",
    "Дальневосточный федеральный округ": "ДФО",
    "Дальневосточный федеральный округ (с 03.11.2018)": "ДФО",
    "Дальневосточный федеральный округ 4)": "ДФО",
    
    "Город федерального значения Севастополь": "Севастополь",
    "г. Севастополь": "Севастополь",
    "г.Севастополь": "Севастополь",
    
    "Крымский федеральный округ": "Республика Крым",
    
    "Ненецкий автономный округ (Архангельская область)": "Ненецкий АО",
    "Ненецкий авт.округ": "Ненецкий АО",
    "в том числе Ненецкий автономный округ": "Ненецкий АО",
    
    "Архангельская область (кроме Ненецкого автономного округа)": "Архангельская область",
    "Архангельская область без данных по Ненецкому автономному округу": "Архангельская область",
    "Архангельская область без авт.округа": "Архангельская область",
    
    "Ханты-Мансийский автономный округ - Югра (Тюменская область)": "ХМАО",
    "Ханты-Мансийский авт.округ": "ХМАО",
    "в том числе Ханты-Мансийский автономный округ - Югра": "ХМАО",
    
    "Ямало-Ненецкий автономный округ (Тюменская область)": "ЯНАО",
    "Ямало-Ненецкий авт.округ": "ЯНАО",
    "в том числе Ямало-Ненецкий автономный округ": "ЯНАО",
    
    "Тюменская область без данных по Ханты-Мансийскому автономному округу - Югре и Ямало-Ненецкому автономному округу": "Тюменская область",
    "Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)": "Тюменская область",
    "Тюменская область без авт.округов": "Тюменская область",
    
    "Чувашская Республика - Чувашия": "Чувашская Республика",
    "Чувашская Республика": "Чувашская Республика",
    
    "Республика Татарстан (Татарстан)": "Татарстан",
    "Республика Татарстан": "Татарстан",
    
    "Республика Адыгея (Адыгея)": "Республика Адыгея",
    
    "Республика Бурятия (до конца 2018 года)": "Республика Бурятия",
    "Забайкальский край (до конца 2018 года)": "Забайкальский край",
    
    "Кемеровская область - Кузбасс": "Кемеровская область",
    
    "Еврейская автономная область": "Еврейская АО",
    "Еврейская авт.область": "Еврейская АО",
    
    "Чукотский автономный округ": "Чукотский АО",
    "Чукотский авт.округ": "Чукотский АО",
    "Республика Северная Осетия - Алания": "Республика Северная Осетия",
    "Республика Северная Осетия-Алания": "Республика Северная Осетия",
    "Российская Федерация без учета новых субъектов (с 01.01.2023)":"Россия"
}

# Применение словаря к каждому датасету
for df in datasets:
    df['region'] = df['region'].replace(region_replace_dict)

all_regions = pd.concat([
    home_average_mortgage_rate_df['region'],
    home_weight_avg_mortgage_rate_df['region'],
    home_average_price_df['region'],
    home_index_price_df['region'],
    avg_monetary_income_people_df['region']
]).unique()

print(all_regions)


['Россия' 'ЦФО' 'Белгородская область' 'Брянская область'
 'Владимирская область' 'Воронежская область' 'Ивановская область'
 'Калужская область' 'Костромская область' 'Курская область'
 'Липецкая область' 'Московская область' 'Орловская область'
 'Рязанская область' 'Смоленская область' 'Тамбовская область'
 'Тверская область' 'Тульская область' 'Ярославская область' 'Москва'
 'СЗФО' 'Республика Карелия' 'Республика Коми' 'Архангельская область'
 'Ненецкий АО' 'Вологодская область' 'Калининградская область'
 'Ленинградская область' 'Мурманская область' 'Новгородская область'
 'Псковская область' 'Санкт-Петербург' 'ЮФО' 'Республика Адыгея'
 'Республика Калмыкия' 'Республика Крым' 'Краснодарский край'
 'Астраханская область' 'Волгоградская область' 'Ростовская область'
 'Севастополь' 'СКФО' 'Республика Дагестан' 'Республика Ингушетия'
 'Кабардино-Балкарская Республика' 'Карачаево-Черкесская Республика'
 'Республика Северная Осетия' 'Чеченская Республика' 'Ставропольский край'
 'ПФО' 'Ре

In [464]:
# даты, в диапазоне которых будут данные
start_date = max(min_date_list)
end_date = min(max_date_list)

# объединеняем данные - "Ключевая ставка", "Инфляция", "Курс доллара"
merged_df = key_rate_df.merge(us_rate_df, on="date", how="outer") \
                       .merge(key_rate_inflation_df, on="date", how="outer")

# Заполняем пропуски
merged_df = merged_df.ffill()

# добавляем новый столбец, для дальнейшего объединения
merged_df['region'] = 'Россия'

print(merged_df)

           date  key_rate   us_rate  inflation  region
0    2013-09-17       5.5       NaN        NaN  Россия
1    2013-09-18       5.5       NaN        NaN  Россия
2    2013-09-19       5.5       NaN        NaN  Россия
3    2013-09-20       5.5       NaN        NaN  Россия
4    2013-09-23       5.5       NaN        NaN  Россия
...         ...       ...       ...        ...     ...
3419 2024-12-26      21.0   99,6125       9.52  Россия
3420 2024-12-27      21.0   99,2295       9.52  Россия
3421 2024-12-28      21.0  100,5281       9.52  Россия
3422 2024-12-29      21.0  101,6797       9.52  Россия
3423 2024-12-30      21.0  101,6797       9.52  Россия

[3424 rows x 5 columns]


In [ ]:
# список остальных датасетов
datasets = [
    home_average_mortgage_rate_df,
    home_weight_avg_mortgage_rate_df,
    home_average_price_df,
    home_index_price_df,
    avg_monetary_income_people_df
]

# Последовательно объединяем остальные датасеты по колонкам 'date'
for df in datasets[1:]:
    merged_df = pd.merge(merged_df, df, on=['date', 'region'], how='outer')

merged_df = merged_df[(merged_df["date"] >= start_date) & (merged_df["date"] <= end_date)]
merged_df = merged_df.ffill()
merged_df = merged_df.sort_values(by='date')

merged_df.to_csv(r'..\data\csv\housing_prices_mortgage_rates.csv', index=False, encoding='utf-8')
merged_df



C:\Users\Mi\AppData\Local\Temp\ipykernel_20304\3915008189.py:15: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df = merged_df.ffill()


,date,key_rate,us_rate,inflation,region,weight_avg_rate,price,index,income
10705,2019-01-01,NaN,NaN,NaN,Алтайский край,9.87,45565.59,106.69,20502.222127
11036,2019-01-01,7.75,"69,4706",5.00,Сахалинская область,9.82,106343.82,129.42,54127.732117
11035,2019-01-01,7.75,"69,4706",5.00,Сахалинская область,9.82,106343.82,109.47,54127.732117
11034,2019-01-01,7.75,"69,4706",5.00,Саратовская область,9.88,38816.62,103.78,19152.330021
11033,2019-01-01,7.75,"69,4706",5.00,Саратовская область,9.88,38816.62,103.44,19152.330021
...,...,...,...,...,...,...,...,...,...
27089,2024-07-01,16.00,"85,7480",8.59,Краснодарский край,7.80,150644.14,110.00,62393.000000
27088,2024-07-01,16.00,"85,7480",8.59,Краснодарский край,7.80,150644.14,104.43,62393.000000
27087,2024-07-01,16.00,"85,7480",8.59,Краснодарский край,7.80,180517.26,110.00,62393.000000
27085,2024-07-01,16.00,"85,7480",8.59,Костромская область,8.83,95204.88,114.60,46395.000000
